# Predicting Movie Rates with Decision Tree

In this report, we are building step by step a decision tree to predict user ratings on movies using the MovieLens dataset, available at http://grouplens.org/datasets/movielens/

The dataset is composed by 3 main files:

**movies.dat:** this file contains information of all movies, in the format < movie id > :: < movie name > :: < pipe separeted list of genders >

**users.dat:** this file contains information of all users, in the format < user id > :: < user gender > :: < user age > :: < ocupation > :: < zip code > 

**ratings.dat:** this file contains information of all ratings, in the format < user id > :: < movie id > :: < rating > :: < timestamp >


## Pre-processing data

The first step in this project is to join the data in the 3 data files into a feature and a label matrices. For the gender informations, for each movie we added a binary variable for each different gender. 

The function **build_features** does all the processing, and returns two matrices, X and y, corresponding to the features and the labels. 

In [1]:
import pandas as pd
from IPython.display import display

def build_features():
    """
    read data from movies, users and rating and return a single pandas dataframe
    of the joined tables, containing all info on ratings
    """
    # movie genres
    genres = [
        'Action',
        'Adventure',
        'Animation',
        'Children\'s',
        'Comedy',
        'Documentary',
        'Drama',
        'Fantasy',
        'Film-Noir',
        'Horror',
        'Musical',
        'Mystery',
        'Romance',
        'Sci-Fi',
        'Thriller',
        'War',
        'Western'
    ]
    # reading movies data
    movies_df = pd.DataFrame(columns=['movie_id'] + genres)
    with open('../data/movies.dat','r') as file:
        
        
        lines = file.readlines()
        for idx in range(len(lines)):
            row = lines[idx].split("::")
            movie_genres = row[-1][:-1].split('|')
            row = row[:-2] # ignore genre and movie name
            for genre in genres:
                row.append(str(genre in movie_genres))
            movies_df.loc[idx] = row
    
    # reading users data
    users_df = pd.read_table('../data/users.dat', 
                    names=['user_id', 'gender', 'age', 'ocupation', 'zip_code'], 
                     sep='::', engine='python')
    users_df['user_id'] = users_df['user_id'].apply(lambda x: str(x))
    users_df['ocupation'] = users_df['ocupation'].apply(lambda x: str(x))
    
    # reading ratings data
    ratings_df = pd.read_table('../data/ratings.dat', 
                    names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                    sep='::', engine='python')
    ratings_df['movie_id'] = ratings_df['movie_id'].apply(lambda x: str(x))
    ratings_df['user_id'] = ratings_df['user_id'].apply(lambda x: str(x))
    
    # join tables
    user_ratings_df = ratings_df.merge(users_df, how='inner', on='user_id')
    features_df = user_ratings_df.merge(movies_df, how='inner', on='movie_id')
    
    # drop unwanted features
    features_df = features_df.drop('timestamp', 1)
    features_df = features_df.drop('zip_code', 1)
    features_df = features_df.drop('user_id', 1)
    features_df = features_df.drop('movie_id', 1)
   
    # print first 5 rows
    display(features_df.head()) 
    
    return features_df
    
data = build_features()

,rating,gender,age,ocupation,Action,Adventure,Animation,Children's,Comedy,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,5,F,1,10,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5,M,56,16,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,4,M,25,12,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,M,25,7,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5,M,50,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Building the Tree


#### Splitting the dataset

Now we are going to build the decision tree. To do so, a good start is to create a method that, given some data rows, a feature and a value, splits the data into two disjoint subsets, according to the value.

In [3]:
def split_df(df, feature, value):
    """
    splits a data frame in two separate subsets: 
    one where all rows have row[feature] >= value for ints or floats and row[feature] == value for strings
    and another one where all rows have row[feature] < value for ints or floats and row[feature] != value for strings 
    """
    if isinstance(value,int) or isinstance(value,float):
        df1 = df.ix[df[feature] >= value]
        df2 = df.ix[df[feature] < value]
    else:
        df1 = df.ix[df[feature] == value]
        df2 = df.ix[df[feature] != value]
        
    return df1, df2


#### Counting labels

We now construct a function to, given a dataframe, count how many rows are there of each label, which will be usefull for determining the nodes of the tree.

In [4]:
def label_counts(df, label_name):
    """
    returns the count of each label value
    """
    return df[label_name].value_counts()
    
print label_counts(data, 'rating')

4    348971
3    261197
5    226310
2    107557
1     56174
Name: rating, dtype: int64


#### Entropy

We need to create a method for evaluating the homogenity of the set. We will use the standard entropy function for that.

In [5]:
from math import log 
log2 = lambda x: log(x)/log(2)

def entropy(df, label_name):
    """
    returns the entropy in a dataset
    """
    s = 0.0
    if len(df.index) == 0:
        return s
    
    counts = label_counts(df, label_name)
    size = len(df.index)
    for _, count in counts.iteritems():
        p = float(count)/size
        s -= p*log2(p)
    return s

print entropy(data, 'rating')

2.1002315644


#### Building the tree

Now we start creating the tree. We use a recursive approach, building a node which maximizes the information gain at each step. Note that a node is represented by two attributes: the name of the feature and a value. These variables are passed as parameters to the  split_df method above written, dividing the dataset into two disjoint subsets.

In [11]:
class node:
    """
    Tree node
    """
    def __init__(self, feature, value, true_node=None, false_node=None):
        self.feature = feature
        self.value = value
        self.true_node = true_node
        self.false_node = false_node

In [27]:
def find_best_criteria(df, label_name):
    """
    build a node with a feature and a value 
    that maximizes information gain for the given dataframe
    """
    # initial variables
    best_criteria = (None, None)
    best_gain = 0.0
    best_sets = (None, None)
    current_entropy = entropy(df, label_name)
    
    # iterate through all possible choices and select the one with the biggest information gain
    for feature in df.columns.values:
        if feature != label_name:
            values = df[feature].unique()
            for value in values:
                df1, df2 = split_df(df, feature, value)
                p = float(len(df1.index))/len(df.index)
                gain = current_entropy-p*entropy(df1, label_name)-(1-p)*entropy(df2, label_name)
                if gain > best_gain:
                    best_gain = gain
                    best_criteria = (feature, value)
                    best_sets = (df1, df2)
    return best_criteria, best_sets

In [29]:
def build_tree(df, label_name, max_depth=None):
    """
    build the decision tree
    """
    if len(df.index) == 0 or max_depth == 0:
        return None
    (feature,value), (df1, df2) = find_best_criteria(df, label_name)
    if feature is None or value is None:
        return None
    new_depth = None if max_depth is None else max_depth-1
    true_node = build_tree(df1, label_name, new_depth)
    false_node = build_tree(df2, label_name, new_depth)
    return node(feature, value, true_node, false_node)

print "Building tree. This may take a while..."
decision_tree = build_tree(data, 'rating', 10)

Building tree. This may take a while...


#### Displaying the tree

Now we build a visualization method for displaying the tree.


In [30]:
def print_tree(tree,indent=''):
    if isinstance(tree.value,int) or isinstance(tree.value,float):
        print str(tree.feature) + ' >= ' + str(tree.value) + ' ?'
    else:
        print str(tree.feature) + ' == ' + str(tree.value) + ' ?'
    if tree.true_node:
        print indent+'T:',
        print_tree(tree.true_node, indent+'  ')
    if tree.false_node:
        print indent+'F:',
        print_tree(tree.false_node,indent+'  ')

print_tree(decision_tree)

Drama == False ?
T: Film-Noir == False ?
  T: Horror == False ?
    T: age >= 35 ?
      T: War == False ?
        T: Sci-Fi == True ?
          T: Children's == True ?
            T: Fantasy == False ?
              T: Action == False ?
                T: Comedy == False ?
                F: age >= 45 ?
              F: Comedy == True ?
                T: age >= 56 ?
                F: Adventure == False ?
            F: Fantasy == False ?
              T: Comedy == True ?
                T: Thriller == False ?
                F: Adventure == True ?
              F: Adventure == True ?
                T: Action == True ?
                F: Comedy == False ?
          F: Action == True ?
            T: Children's == True ?
              T: Animation == False ?
                T: Fantasy == True ?
                F: gender == M ?
              F: Western == False ?
                T: Thriller == False ?
                F: Adventure == True ?
            F: Comedy == False ?
            